# BIOINF529 Homework #1 - Winter 2022
This homework is worth **10% of your final grade**.

The exam is due before the next course module begins as enforced by Canvas.

## Coding by Contract
We (the Instructors) promise a fair, impartial, and objective means of grading such that you (the Students) follow the tenets of Coding by Contract:
1. You must not modify/delete any of the existing code in this document (besides the `pass` statements)
* Your functions must use the function signatures as written
* Your functions must return/print the expected results (as written)

If these are followed correctly, your submission should be compatible with the automated testing suite. Therefore, the more tests your code passes, the less scrutiny your code will be under by our review. We do not care *how* you get there, just that you get there *correctly*.

## Submission
Please rename this notebook to **homework1_uniqname.ipynb** for submission. 

For example:
> `homework1_apboyle.ipynb`

We will *only* grade the most recent submission of your exam.

## Late Policy
Each submission will receive a **25%** penalty per day (up to three days) that the assignment is late.

After that, the student will receive a **0** for the homework.

## Academic Honor Code
You may consult with others. However, all answers must be your own and code comparison software will be used to enforce this rule. You are allowed to ask questions at office hours but the answers given will be high-level/conceptual in nature.


---
# de Bruijn graphs

In class we discussed an error-free creation of a de Bruijn graph. This can be applied to genome assembly from high-throughput sequencing of short reads. Reads can be considered randomly drawn from the genome and can be compiled to form a de Bruijn graph. Overall, edge weights should correspond to the average number of that particular k-mer in the genome. This can be used to identify repeated elements in the genome. In reality however, short read sequencing technologies have an inherent error rate (0.2% - 0.5% for Illumina sequencing or up to 10% for Oxford Nanopore sequencing). We can use longer k-mers to our advantage to elimnate these errors and build a full genome assembly without problems.

For example, given a k of 20, there are $4^{20}$ possible k-mers that are possible while only ~ 3 billion 20-mers in the human genome. The vast majority of errors will thus yield a k-mer that does not exist elsewhere in the genome and so it can be ignored.

In this problem, we will be reading in data from a fasta file of sequenceing from our genome sequence, splitting these in to k-lenth k-mers, correcting for sequencing errors in our de Bruijn assembly, building a de Bruijn graph, and then outputting the sequence. A pseudo-code implementation of this is below:

```
error_correcting_deBruijn():
    input reads from fasta file
    count all k-mers in the file
    for all k-mers with frequency <= threshold:
        if there is a k-mer with hamming distance of 1 from k-mer and 
          frequency > threshold and
          the new kmer results is a new adjacent kmer that exists 
              in our k-mers from the fasta file (unless this is the end of the read)
          then use the similar kmer 
        otherwise, use original k-mer
    buld de Bruijn graph with cleaned k-mer list
    print final walk of graph
```

You are expected to implement and will be graded on the following functions:
* `error_correcting_deBruijn`
* `hamming_distance`
* `count_kmers`
* `clean_read`
* `replace_kmer`.

Note that all of the lines in the FASTA file should be used to generate the de Bruijn graph and a single string of the eulerian walk is expected as the output.

Implement as defined in the code below and according to the concept of **'coding by contract'** as discussed in class. If your functions do not take as input or provide as output the variables that we define, you will not receive credit.

In [204]:
# These should be the only imports you need to complete this question
from homework1 import DeBruijnGraph, get_fasta
from collections import Counter
import random

In [205]:
def hamming_distance(sequence1, sequence2): 
    ''' Function to calculate Hamming distance between two sequences
    Hamming distance is defined as the total number of changes required
     to make the sequences identical, or alternatively the total number
     of bases that do not match between the sequences.
    
    Args: 
        sequence1 (str): k-mer to be compared
        sequence2 (str): k-mer to be compared

    Returns:
        distance (int): hamming distance between the two sequences
    
    >>> hamming_distance("ATGCTAT", "ATGCCAG") #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    2
    '''
    distance_counter = 0
    #put all the sequences at each index together in a tuple for comparison
    #using the two already given arguments. 
    result = zip(sequence1, sequence2)
    #use for loop to compare each element at each respective index to see if they are equal (no difference) 
    for seq_1, seq_2 in result:
        #if there is a difference, add to the counter that is originally set to 0. 
        if seq_1 != seq_2:
            distance_counter += 1
    return distance_counter


### My sanity checks/work I performed to build the hamming_distance() function is below. 

In [181]:
#All my sanity check for hamming dist. 
hamming_distance("T", "X")

1

In [182]:
#sanity check for hamming dist. 
hamming_distance("ATGCTAT", "ATGCCAG")

2

In [64]:
#another sanity check. 
hamming_distance("TTCCCCATT", "TTTTCCAAA")

4

In [16]:
#santiy check for hamming dist. 
seq_1 = "AATTCCGT"
seq_2 = "AATGCGGT" 

count = 0

result = zip(seq_1, seq_2)
for i, k in result:
    if i != k:
        count += 1
print(count)

2


In [185]:
def count_kmers(fasta_file, k):
    """ This function builds Counter of all k-mers in a fasta file
        
        Args:
            fasta_file (str): input fasta file of sequence reads
            k (int): k-mer length

        Returns:
            count (Counter): count of all k-mers
        
        Example:
        # Note this may fail for you because the dict order is not stable - just make sure that your output is appropriate
        >>> count_kmers('data/test.fa', 1) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
        Counter({'T': 19, 'A': 17, 'C': 14, 'G': 4})
    """
    
    kmer_counter = Counter()
    for name, seq in get_fasta(fasta_file):    
        for pos in range(len(seq)-k+1): #getting an index for the k-length kmer throughout the seq. 
        #looking for every position (pos) 
            kmer = seq[pos:pos+k] #now indexing throughout the seq to get the right kmer. 
            if kmer in kmer_counter:
                kmer_counter.update([kmer]) #so if the kmer is already in the counter still update the counter. 
                #need to index the kmer since the kmer is an index of the entire sequence at certain positions (aka pos)
            else:
                kmer_counter.update([kmer]) #if it is a brand new kmer than obviously update the counter too. 
            
    return kmer_counter


### My sanity checks/work I performed to build the count_kmers() function is below.

In [186]:
#All my Sanity checks/work for count_kmers function:
count_kmers("data/test.fa", 1)

Counter({'C': 14, 'A': 17, 'T': 19, 'G': 4})

In [187]:
count_kmers("data/test.fa", 2)

Counter({'CA': 13, 'AT': 16, 'TC': 12, 'CT': 1, 'TT': 2, 'TG': 4, 'GA': 4})

In [97]:
count_kmers("data/exam_test.fa", 1)

Counter({'G': 102, 'C': 96, 'A': 60, 'T': 48})

In [71]:
#more sanity checks/work for this function.
test_1 = get_fasta("data/test.fa")

In [72]:
s = list(test_1)
print(s)

[('seq1', 'CATCATCATCATCATCATCATCATCTT'), ('seq1', 'CATCATCATCATCATTGATGATGATGA')]


In [73]:
#sanity checks and work for count_kmers function 
test = Counter("CATCATCATCATCATCATCATCATCTTCATCATCATCATCATTGATGATGATGA")
test

Counter({'C': 14, 'A': 17, 'T': 19, 'G': 4})

In [74]:
seq_test = "AATTCCTT"
len(seq_test)

8

In [75]:
#testing to make sure I'm getting the correct k-mer length out. 
def kmer_length(seq, k):
    kmer_list = [] #empty list to get my 
    for pos in range(len(seq)-k+1):
        kmer = seq[pos:pos+k]
        kmer_list.append(kmer)
    return kmer_list

In [76]:
#now trying to incorporate the Counter into this 
def kmer_length(seq, k):
    kmer_counter = Counter() #got this from hw in bioinf575
    
    for pos in range(len(seq)-k+1): #getting an index for the k-length kmer throughout the seq. 
        #looking for every i position. 
        kmer = seq[pos:pos+k] #now indexing throughout the seq to get the right kmer. 
        if kmer in kmer_counter:
            kmer_counter.update([kmer])#so if the kmer is already in the counter just add 1 to it. 
        else:
            kmer_counter.update([kmer])
    return kmer_counter

In [77]:
#more testing
test24 = kmer_length("AATTTCCTTT", 2)
test24

Counter({'AA': 1, 'AT': 1, 'TT': 4, 'TC': 1, 'CC': 1, 'CT': 1})

In [78]:
#more testing 
test_2 = kmer_length("AATTTCCTTT", 3)

In [79]:
kmer_list = test_2.keys()

In [80]:
kmer_list

dict_keys(['AAT', 'ATT', 'TTT', 'TTC', 'TCC', 'CCT', 'CTT'])

In [81]:
kmer_length("AATTTCCTTT", 1)

Counter({'A': 2, 'T': 6, 'C': 2})

In [82]:
#another test to make sure I have this counter thing right.
word = "mississippi"
counter = {}
for letter in word:
    if letter not in counter:
        counter[letter] = 0
    counter[letter] += 1  
print(counter)

{'m': 1, 'i': 4, 's': 4, 'p': 2}


In [83]:
#another test when first building kmer_count() function. 
def kmer_length(seq, k):
    kmer_counter = Counter() #got this from hw in bioinf575
    for pos in range(len(seq)-k+1): #getting an index for the k-length kmer throughout the seq. 
        #looking for every i position. 
        kmer = seq[pos:pos+k] #now indexing throughout the seq to get the right kmer. 
        if kmer in kmer_counter:
            kmer_counter[kmer] += 1 #so if the kmer is already in the counter just add 1 to it. 
        else:
            kmer_counter[kmer] = 1
    return kmer_counter

In [84]:
#final test with my kmer_length function and I get same output as in the homework function doctest. 
kmer_length("CATCATCATCATCATCATCATCATCTTCATCATCATCATCATTGATGATGATGA", 1)

Counter({'C': 14, 'A': 17, 'T': 19, 'G': 4})

In [190]:
def replace_kmer(kmer_count, kmer, threshold): #this one has the threshold. 
    """ This function replaces a random k-mer that is within
            hamming distance of 1 and frequency > threshold,
            otherwise returns original k-mer
        
        Args:
            kmer_count (Counter): count of all k-mers
            kmer (str): k_mer being replaced
            threshold (int): threshold for k-mer replacement

        Returns:
            kmer (str): new kmer
        
        Example:
        >>> replace_kmer(count_kmers('data/test.fa', 1), 'X', 18) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
        'T'
    """
    
    kmer_dict = kmer_count #Counter Dictionary 
    kmer_key_list = list(kmer_dict) #list of keys 
    # print(kmer_key_list)
    kmer_list_index = list(range(len(kmer_key_list)))
    kmer_shuffle_index_list = random.sample(kmer_list_index, len(kmer_list_index)) #want the number of 
    for kmer_shuffle_index in kmer_shuffle_index_list:
        # print(kmer_shuffle_index)
        kmer_random = kmer_key_list[kmer_shuffle_index]
        count_freq = kmer_dict[kmer_random]
        ham_dist = hamming_distance(str(kmer), str(kmer_random))

        if count_freq > threshold and ham_dist == 1: #if conditions are met 
            kmer_final = kmer_random
            break #get out of the for loop once we have met both of these conditions.    
        else:
            kmer_final = kmer

    return kmer_final

### My sanity checks/work I performed to build the replace_kmer() function is below.

In [191]:
replace_kmer(count_kmers('data/test.fa', 1), 'X', 18)

'T'

In [192]:
replace_kmer(count_kmers("data/exam_test.fa", 1), "X", 100)

'G'

In [110]:
kmer_count = count_kmers('data/test.fa', 1)
sum(list(kmer_count.values()))

54

In [111]:
kmer_dict = count_kmers('data/test.fa', 1) #Counter Dictionary 
kmer_key_list = list(kmer_dict) #list of keys
# print(range(0:(len(kmer_key_list)-1)))

kmer_list_index = list(range(len(kmer_key_list)-1))
print(kmer_list_index)
type(kmer_list_index)

[0, 1, 2]


list

In [112]:
random.shuffle(kmer_list_index)
print(kmer_list_index)

[2, 0, 1]


In [113]:
test1 = random.sample(kmer_list_index, len(kmer_list_index))
print(test1)

[0, 1, 2]


In [114]:
range(5)

range(0, 5)

In [115]:
dict_test = {"AAT":1, "TTT":2, "CCG":3}
dict_list = list(dict_test)
random_kmer = random.choice(dict_list)
# random_kmer
frequency = dict_test[random_kmer]
frequency

3

In [119]:
air = {"AAT": 1,"CCT": 2,"GGC": 3, "TTA": 4} #counter_dict 
list(air)  #get the keys 

['AAT', 'CCT', 'GGC', 'TTA']

In [120]:
type(list(air))

list

In [122]:
def test_1(kmer_count):
    kmer_dict = kmer_count
    kmer_dict_keys = list(kmer_dict)
    for random_kmer in random.choice(kmer_dict_keys):
        frequency = kmer_dict[random_kmer]
        return random_kmer

In [123]:
test_1(count_kmers("data/test.fa", 1))

'A'

In [124]:
def test_2(kmer_count, threshold):
    kmer_dict = kmer_count
    kmer_dict_keys = list(kmer_dict)
    for random_kmer in random.choice(kmer_dict_keys):
        frequency = kmer_dict[random_kmer]
        if frequency > threshold:
            return frequency

In [125]:
test_2(count_kmers("data/test.fa", 1), 6)

14

In [195]:
def clean_read(read, k, kmer_count, threshold):
    """ This function cleans a read by replacing all k-mers within the read
            using the replace_kmer function 
            
        Args:
            read (str): the full-length read from the fasta file
            k (int): k-mer length
            kmer_count (Counter): count of all k-mers
            threshold (int): threshold for k-mer replacement

        Returns:
            new_read (str): read with errors corrected
        
        Example:
        >>> clean_read('ATG', 1, count_kmers('data/test.fa', 1), 18) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
        'TTT'
    """
    
    for pos in range(len(read)-k+1):
        read_kmer = read[pos:pos+k]
        if kmer_count[read_kmer] <= threshold: 
        #now going to call the replace_kmer() to return the final kmer. 
            kmer_replace = replace_kmer(kmer_count, read_kmer, threshold)
            read = read.replace(read_kmer, kmer_replace) #replacing a variable, need to always change the original variable. 
            
    return read

### My sanity checks/work I performed to build the clean_read() function is below.

In [196]:
clean_read("ATG", 1, count_kmers("data/test.fa", 1), 18)

'TTT'

In [197]:
clean_read("AACCGGTT", 1, count_kmers("data/exam_test.fa", 1), 50)

'AACCGGAA'

In [152]:
clean_read("ATG", 1, count_kmers("data/test.fa", 1), 15)

'ATT'

In [146]:
read = "ATG"
k =1
list(range(len(read)))

[0, 1, 2]

In [76]:
for pos in range(len(read)-k+1):
        read_kmer = read[pos:pos+k]
        print(read_kmer)

A
T
G


In [86]:
#I think this is what they want now for clean_read() function 
seq = "ATGTGT"
type(seq)
seq_index = seq[0:3]
seq_2 = "AGC"
seq_2
new_seq = seq.replace(seq_index, seq_2)
new_seq

'AGCTGT'

In [153]:
read = "ATG"
read_list = list(read)
len(read_list)

3

In [93]:
read_list[2] = kmer_new
final_read = "".join(read_list)
final_read

'ATACT'

In [200]:
def error_correcting_deBruijn(input_file, k, threshold, seed=42):
    """ This function builds a de Bruijn graph but uses error correction on the input
          This fasta file can contain multiple sequence reads and please note that 
          individual fasta entries should not be concatenated.
            
        Args:
            input_file (str): input fasta file of sequence reads
            k (int): k-mer length
            threshold (int): threshold for k-mer replacement
            seed (int): seed for eulerian walk

        Returns:
            walk (str): eulerian walk of the de Bruijn graph
        
        Example:
        >>> error_correcting_deBruijn('data/test.fa', 3, 1, 1) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
        'CAT...'
    """  

    for name, seq in get_fasta(input_file):
        #for every sequence in the fasta file, I need to push through the clean_read() function. 
        reads_cleaned = clean_read(seq, k, count_kmers(input_file, k), threshold)
        #now I need to put each cleaned read into the graph. 
        graph = DeBruijnGraph(reads_cleaned, k)
        #need to build the actual graph first. 
        graph.build_debruijn_graph(reads_cleaned, k)
    #now need to print the walk out. 
    eul_walk = graph.print_eulerian_walk(seed = seed)
    #now need to join the walks. 
    final_eul_walk = eul_walk[0] + ''.join(map(lambda x: x[-1], eul_walk[1:]))

    return final_eul_walk
        

### My sanity checks/work I performed to build the error_correcting_deBruijn() function is below.

In [201]:
error_correcting_deBruijn("data/test.fa", 3, 1, 1)

'CATCATCATCATGATCATCATCATCATCATCATCATCATGATCATCATCAT'

In [202]:
error_correcting_deBruijn("data/exam_test.fa", 3, 1, 1)

'GATGGGCCAGCTGGGAGCAGATGCCCAGGGGGAACCCTTGAGGAAGAACCCTGGGAGGGCAGGCAGGTGAGACCGAGGGGGTGGGAAGGCTGCTTGGAGAAGCCCCCTGAGGGGTGATGCCAGGATGAGGAGAGGAGGAAGCACCTTCCCGACCTCCACCCTGCTTTTTT'

In [173]:
string = ""
type(string)
string.join("A")

'A'

In [174]:
for name, seq in get_fasta("data/test.fa"):
    print(seq)
    seq_test = str(seq)
    print(seq_test)     

CATCATCATCATCATCATCATCATCTT
CATCATCATCATCATCATCATCATCTT
CATCATCATCATCATTGATGATGATGA
CATCATCATCATCATTGATGATGATGA


In [175]:
type(seq_test)

str

In [176]:
seq = ["AATT", "GGCC"]
# type(seq)
seq_str = str(seq)
# type(seq_str)
# print(seq_str)
sum_seq = "".join(seq_str)
print(sum_seq)
# sum_seq = "".join(sum)
# print(seq)

['AATT', 'GGCC']


---
# Testing your code
**Instructions:** If you want a to perform a cursory check of your code (for debugging purposes), do the following:
1. Run all cells that the function(s) you want to test depends on
* Run the cell(s) you want to test
* Run the cell below

**Note:** This tests *only* the test cases that are present in the docstrings. Therefore, a passing test does not mean your program(s) work for *all possible* test cases. It is up to you to determine edge-cases that we may consider during grading.

In [203]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=5)